In [1]:
!pip install -U gensim==4.0.0
!pip install python-Levenshtein
!pip install textattack

     |████████████████████████████████| 23.9MB 131kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149799 sha256=778abcd960c08ba9daeace6414350d248480c2adf98f33c1c3f3dd51181607bd
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 358kB 19.1MB/s 
     |████████████████████████████████| 20.1MB 167kB/s 
     |████████████████████████████████| 337kB 57.8MB/s 
     |████████████████████████████████| 225kB 57.4MB/s 
     |████████████████████████████████| 778kB 48.2MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 2.1MB 56

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("./drive/MyDrive/models/fasttext.model") # Change this location

In [ ]:
#model1.wv.most_similar('cow')

[('dog', 0.7193913459777832),
 ('squirrel', 0.696162760257721),
 ('leopardus', 0.6945451498031616),
 ('prionailurus', 0.688106894493103),
 ('albino', 0.6880608201026917),
 ('bird', 0.6836060285568237),
 ('eared', 0.6803243160247803),
 ('rhinoceros', 0.6769237518310547),
 ('hyena', 0.6756084561347961),
 ('badger', 0.6752869486808777)]

In [4]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return a word containing the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:09<00:00, 48.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpvfs1n1hn.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [35]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Loading datasets dataset yelp_polarity, split test.


Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581. Subsequent calls will reuse this data.


In [36]:
data = []
for i in range(100):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
for i in range(-100,-1):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
data.sort(key = lambda x: len (x[0]))

In [ ]:
print(model1.wv.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]


In [6]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]
# # We'll use the Greedy search method
search_method = GreedyWordSwapWIR("weighted-saliency")
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [38]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 15
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")







  0%|          | 0/15 [00:00<?, ?it/s]







  7%|▋         | 1/15 [00:00<00:04,  2.91it/s]





[Succeeded / Failed / Total] 1 / 0 / 1:   7%|▋         | 1/15 [00:00<00:04,  2.84it/s]







[Succeeded / Failed / Total] 1 / 0 / 1:  13%|█▎        | 2/15 [00:00<00:06,  2.04it/s]





[Succeeded / Failed / Total] 1 / 1 / 2:  13%|█▎        | 2/15 [00:00<00:06,  2.01it/s]







[Succeeded / Failed / Total] 1 / 1 / 2:  20%|██        | 3/15 [00:01<00:05,  2.23it/s]





[Succeeded / Failed / Total] 2 / 1 / 3:  20%|██        | 3/15 [00:01<00:05,  2.22it/s]







[Succeeded / Failed / Total] 2 / 1 / 3:  27%|██▋       | 4/15 [00:01<00:05,  2.18it/s]





[Succeeded / Failed / Total] 3 / 1 / 4:  27%|██▋       | 4/15 [00:01<00:05,  2.17it/s]

...







[Succeeded / Failed / Total] 12 / 1 / 17: : 17it [00:10,  1.70it/s]







[Succeeded / Failed / Total] 13 / 1 / 18: : 18it [00:11,  1.61it/s]







[Succeeded / Failed / Total] 13 / 1 / 18: : 19it [00:12,  1.56it/s]





[Succeeded / Failed / Total] 14 / 1 / 19: : 19it [00:12,  1.56it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 78.95% |
| Accuracy under attack:        | 5.26%  |
| Attack success rate:          | 93.33% |
| Average perturbed word %:     | 14.74% |
| Average num. words per input: | 17.63  |
| Avg num queries:              | 34.6   |
+-------------------------------+--------+


textattack: Attack time: 12.196712970733643s


BERT:

In [10]:
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import WordSwapMaskedLM

        # Swap words with their inflections
transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=48)
        #
        # Don't modify the same word twice or stopwords.
        #
constraints = [RepeatModification(), StopwordModification()]

        # "We only take ε percent of the most important words since we tend to keep
        # perturbations minimum."
        #
        # [from correspondence with the author]
        # "Word percentage allowed to change is set to 0.4 for most data-sets, this
        # parameter is trivial since most attacks only need a few changes. This
        # epsilon is only used to avoid too much queries on those very hard samples."
constraints.append(MaxWordsPerturbed(max_percent=0.4))

        # "As used in TextFooler (Jin et al., 2019), we also use Universal Sentence
        # Encoder (Cer et al., 2018) to measure the semantic consistency between the
        # adversarial sample and the original sequence. To balance between semantic
        # preservation and attack success rate, we set up a threshold of semantic
        # similarity score to filter the less similar examples."
        #
        # [from correspondence with author]
        # "Over the full texts, after generating all the adversarial samples, we filter
        # out low USE score samples. Thus the success rate is lower but the USE score
        # can be higher. (actually USE score is not a golden metric, so we simply
        # measure the USE score over the final texts for a comparison with TextFooler).
        # For datasets like IMDB, we set a higher threshold between 0.4-0.7; for
        # datasets like MNLI, we set threshold between 0-0.2."
        #
        # Since the threshold in the real world can't be determined from the training
        # data, the TextAttack implementation uses a fixed threshold - determined to
        # be 0.2 to be most fair.
use_constraint = UniversalSentenceEncoder(
            threshold=0.2,
            metric="cosine",
            compare_against_original=True,
            window_size=None,
        )
constraints.append(use_constraint)
        #
        # Goal is untargeted classification.
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # "We first select the words in the sequence which have a high significance
        # influence on the final output logit. Let S = [w0, ··· , wi ··· ] denote
        # the input sentence, and oy(S) denote the logit output by the target model
        # for correct label y, the importance score Iwi is defined as
        # Iwi = oy(S) − oy(S\wi), where S\wi = [w0, ··· , wi−1, [MASK], wi+1, ···]
        # is the sentence after replacing wi with [MASK]. Then we rank all the words
        # according to the ranking score Iwi in descending order to create word list
        # L."
search_method = GreedyWordSwapWIR(wir_method="unk")

attack1 = Attack(goal_function, constraints, transformation, search_method)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [39]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 15
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")







  0%|          | 0/15 [00:00<?, ?it/s]







  7%|▋         | 1/15 [00:01<00:14,  1.07s/it]





[Succeeded / Failed / Total] 1 / 0 / 1:   7%|▋         | 1/15 [00:01<00:15,  1.07s/it]







[Succeeded / Failed / Total] 1 / 0 / 1:  13%|█▎        | 2/15 [00:47<05:09, 23.79s/it]





[Succeeded / Failed / Total] 2 / 0 / 2:  13%|█▎        | 2/15 [00:47<05:09, 23.80s/it]







[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 3/15 [00:48<03:14, 16.25s/it]





[Succeeded / Failed / Total] 3 / 0 / 3:  20%|██        | 3/15 [00:48<03:14, 16.25s/it]







[Succeeded / Failed / Total] 3 / 0 / 3:  27%|██▋       | 4/15 [00:50<02:19, 12.69s/it]





[Succeeded / Failed / Total] 4 / 0 / 4:  27%|██▋       | 4/15 [00:50<02:19, 12.69s/it]

...







[Succeeded / Failed / Total] 12 / 0 / 16: : 17it [01:58,  6.99s/it]





[Succeeded / Failed / Total] 13 / 0 / 17: : 17it [01:58,  7.00s/it]







[Succeeded / Failed / Total] 13 / 0 / 17: : 18it [02:01,  6.74s/it]





[Succeeded / Failed / Total] 14 / 0 / 18: : 18it [02:01,  6.74s/it]







[Succeeded / Failed / Total] 14 / 0 / 18: : 19it [02:03,  6.51s/it]





[Succeeded / Failed / Total] 15 / 0 / 19: : 19it [02:03,  6.51s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 78.95% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 12.52% |
| Average num. words per input: | 17.63  |
| Avg num queries:              | 110.8  |
+-------------------------------+--------+


textattack: Attack time: 123.67272591590881s
